### 1. Connecting to Redis

We are using Redis as a key-value database to store session data for an e-commerce user. Redis is chosen because it provides low-latency, in-memory data storage, making it ideal for session management. Its ability to handle large volumes of data and provide fast access is important for user sessions that need to be retrieved and updated quickly.

#### Key Advantages of Redis for Session Management:
- **Fast Read/Write**: Redis operates in memory, which makes data retrieval and updates extremely fast.
- **Persistence**: Redis supports data persistence, meaning session data can survive restarts if configured appropriately.
- **Scalability**: Redis can be scaled horizontally with Redis clusters, making it capable of handling large amounts of session data.



### Connecting to Redis

In [1]:

import redis
import json
import os
from dotenv import load_dotenv

load_dotenv()

# Configuration for Redis connection
redis_host = os.getenv('REDIS_HOST')
redis_port = int(os.getenv('REDIS_PORT'))  # Cast to int if necessary
redis_password = os.getenv('REDIS_PASSWORD')

# Initialize Redis client
r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

# Test Connection
try:
    r.ping()  # Check if Redis server is alive
    print("Connected to Redis successfully!")
except redis.ConnectionError as e:
    print(f"Failed to connect to Redis: {e}")


Connected to Redis successfully!



### 2. Managing Sessions in Redis
For e-commerce applications, managing user sessions efficiently is critical. Redis is a suitable choice because it can store session data like user preferences, shopping cart items, and session metadata in a fast and scalable manner. Here's how we implement CRUD operations for session management:


### 3. Creating a Session (Storing Shopping Cart Data)
This function creates a session for an e-commerce user, storing their session data and shopping cart in Redis. Redis's setex method is used to store the session with an expiration time (e.g., 1 hour).

In [2]:
def create_session(session_id, user_data, expiration_seconds=3600):
    """
    Create a session for a user and store session data in Redis.
    
    :param session_id: Unique session identifier (e.g., user ID)
    :param user_data: Dictionary containing user-specific data (e.g., shopping cart)
    :param expiration_seconds: Session expiration time in seconds (default is 1 hour)
    """
    r.setex(session_id, expiration_seconds, json.dumps(user_data))  # Store session with expiration
    print(f"Session {session_id} created with expiration in {expiration_seconds} seconds.")

# Example Usage
user_data = {"user_id": "U001", "name": "Rohith", "preferences": {"theme": "dark", "browser": "Edge"}}
create_session("task1", user_data)


Session task1 created with expiration in 3600 seconds.


### 4. Retrieving Session Data
This function retrieves the session data for a user based on their session ID. If the session exists, it is returned; otherwise, a message is printed that the session does not exist.

In [3]:
def get_session_data(session_id):
    """
    Retrieve session data from Redis.
    
    :param session_id: Unique session identifier
    :return: Session data as a Python dictionary or None if session does not exist
    """
    session_data = r.get(session_id)  # Get session data from Redis
    if session_data:
        return json.loads(session_data)  # Convert JSON string to dictionary
    else:
        print(f"Session {session_id} does not exist.")
        return None

# Example Usage
session_data = get_session_data("task1")
print("Retrieved Session Data:", session_data)


Retrieved Session Data: {'user_id': 'U001', 'name': 'Rohith', 'preferences': {'theme': 'dark', 'browser': 'Edge'}}


### 5. Updating Session Data
This function updates the existing session data in Redis. It checks if the session exists, and if so, it updates the session data.

In [4]:
def update_session_data(session_id, new_user_data):
    """
    Update existing session data.
    
    :param session_id: Unique session identifier
    :param new_user_data: Dictionary containing updated user-specific data
    """
    if r.exists(session_id):  # Check if session exists in Redis
        r.set(session_id, json.dumps(new_user_data))  # Update session data
        print(f"Session {session_id} updated.")
    else:
        print(f"Session {session_id} does not exist.")

# Example Usage
updated_user_data = {"user_id": "U001", "name": "Alice", "preferences": {"theme": "light", "browser": "chrome"}}
update_session_data("task1", updated_user_data)


Session task1 updated.


### 6. Deleting a Session
This function deletes a session from Redis. It checks if the session exists before attempting to delete it.

In [5]:
def delete_session(session_id):
    """
    Delete a session by session_id.
    
    :param session_id: Unique session identifier
    """
    if r.delete(session_id):  # Delete session from Redis
        print(f"Session {session_id} deleted.")
    else:
        print(f"Session {session_id} does not exist.")

# Example Usage
delete_session("task1")


Session task1 deleted.



### 7. Justification for Redis in Session Management
* **High Performance**: Redis provides extremely fast data access because it is in-memory.
* **Built-in Expiration**: Redis allows setting expiration times for session data, making it ideal for temporary storage like sessions.
* **Scalability**: Redis can scale horizontally with clustering, meaning it can handle a high volume of concurrent users.
* **Atomic Operations**: Redis supports atomic operations, ensuring that session data is updated consistently.
* 
### 8. Handling Large Amounts of Data
Redis can handle large amounts of data (even more than 10TB) by using sharding across multiple nodes. For scenarios requiring persistence and fault tolerance, Redis provides:

* **Persistence Options**: Redis supports both RDB (point-in-time snapshots) and AOF (append-only file) persistence modes.
* **Clustering**: Redis Clusters can scale horizontally by partitioning data across multiple nodes, each handling a subset of the data.
**Managing Data Performance:**
* **Memory Efficiency**: Use appropriate data types and structures (e.g., Redis Hashes) to reduce memory usage.
* **Eviction Policies**: Redis allows setting eviction policies (like LRU) to free up memory by removing the least recently used keys.

### 9. Testing the Session Management System
Below are test cases to validate our session management implementation:

1. **Create Session**: Create a new session for a user.
2. **Retrieve Session**: Retrieve the stored session data to ensure it was saved correctly.
3. **Update Session**: Update the session data and verify that the changes were applied.
4. **Delete Session**: Delete the session and confirm that it no longer exists.

In [6]:
# Test Case 1: Create and Retrieve Session
user_data = {"user_id": "U002", "name": "John", "preferences": {"theme": "dark", "browser": "Firefox"}}
create_session("task2", user_data)

session_data = get_session_data("task2")
assert session_data['name'] == 'John', "Test Case 1 Failed: Data mismatch"

# Test Case 2: Update Session
updated_user_data = {"user_id": "U002", "name": "John", "preferences": {"theme": "light", "browser": "Safari"}}
update_session_data("task2", updated_user_data)

updated_session_data = get_session_data("task2")
assert updated_session_data['preferences']['theme'] == 'light', "Test Case 2 Failed: Update failed"

# Test Case 3: Delete Session
delete_session("task2")
assert get_session_data("task2") is None, "Test Case 3 Failed: Deletion failed"

print("All test cases passed!")


Session task2 created with expiration in 3600 seconds.
Session task2 updated.
Session task2 deleted.
Session task2 does not exist.
All test cases passed!
